In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [1]:
!git clone https://github.com/casasvoley/Grape-harvest-counting

Cloning into 'Grape-harvest-counting'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 162 (delta 4), reused 94 (delta 1), pack-reused 60
Receiving objects: 100% (162/162), 294.07 MiB | 32.99 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (138/138), done.


In [ ]:
import os
import cv2
import albumentations as A

In [ ]:
def read_annotations_yolo(annotation_txt):
    with open(annotation_txt, "r") as f:
        lines = f.readlines()
    
    annotations = []
    for line in lines:
        annotations.append([float(i) for i in line.strip().split(" ")])
        
    return annotations

In [ ]:
def save_annotations_yolo(annotations, output_file):
    with open(output_file, "w") as f:
        for annotation in annotations:
            line = " ".join(str(i) for i in annotation)
            f.write(line + "\n")

In [ ]:
def apply_transformation(img, annotations, transform):
    labels = [x[0] for x in annotations]
    bboxes = [x[1:] for x in annotations]
    
    result = transform(image=img, bboxes=bboxes, labels=labels)
    transformed_img = result["image"]
    transformed_annotations = [
        [label, *box] for label, box in zip(result['labels'], result['bboxes']) if box[2] > 0 and box[3] > 0
    ]
    
    return transformed_img, transformed_annotations


In [ ]:
def augment_dataset(input_folder, output_folder, num_output_images=1000):
    transform = A.Compose([
        A.Resize(512, 512),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Rotate(limit=10, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(0.3, 0.5), p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.RandomScale(scale_limit=0.2, p=0.5),
        A.GaussNoise(var_limit=(10, 50), mean=0, p=0.5),
        A.GaussianBlur(p=0.5),
        A.MotionBlur(p=0.5),
        A.CLAHE(clip_limit=4.0, p=0.5),
        A.Cutout(num_holes=8, max_h_size=16, max_w_size=16, p=0.5),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['labels'], min_area=0.001, min_visibility=0.001))


    transform_original = A.Compose([
        A.Resize(512, 512)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['labels'], min_area=0.001, min_visibility=0.001))


    os.makedirs(output_folder, exist_ok=True)

    all_files = os.listdir(input_folder)
    img_files = [f for f in all_files if f.endswith(".jpg")]
    num_files = len(img_files)

    for img_name in img_files:
        img_path = os.path.join(input_folder, img_name)
        txt_path = os.path.join(input_folder, f"{img_name[:-4]}.txt")

        # Checks if the .txt file exists before continuing
        if not os.path.exists(txt_path):
            continue
        
        img = cv2.imread(img_path)
        annotations = read_annotations_yolo(txt_path)

        # Resize the original image and its annotations
        img_resized, annotations_resized = apply_transformation(img, annotations, transform_original)

        # Save the resized image and its annotations in the output folder
        cv2.imwrite(os.path.join(output_folder, img_name), img_resized)
        save_annotations_yolo(annotations_resized, os.path.join(output_folder, f"{img_name[:-4]}.txt"))
       
        # Loop that generates augmented images and saves them in the output folder
        for i in range(num_output_images // num_files):
            img_aug, annotations_aug = apply_transformation(img, annotations, transform)

            new_img_name = f"{img_name[:-4]}_aug_{i}.jpg"
            new_txt_name = f"{img_name[:-4]}_aug_{i}.txt"

            cv2.imwrite(os.path.join(output_folder, new_img_name), img_aug)
            save_annotations_yolo(annotations_aug, os.path.join(output_folder, new_txt_name))

In [ ]:

input_folder = "/content/Grape-harvest-counting/datasets/grapes/train"
output_folder = "/content/Grape-harvest-counting/datasets/grapes_augmented/train"
num_output_images = 300

augment_dataset(input_folder, output_folder, num_output_images)

In [ ]:
import cv2
import os
import albumentations as A

def improve_brightness_saturation(input_image_path, output_image_path):
    
    transform = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(0.3, 0.5), p=1.0),
        A.HueSaturationValue(sat_shift_limit=40, p=1.0),  # Aumenta la diferencia de color
        A.ImageCompression(quality_lower=60, quality_upper=100, p=1.0),  # Mejora los detalles de la imagen

    ])

    img = cv2.imread(input_image_path)

    img_transformed = transform(image=img)["image"]

    cv2.imwrite(output_image_path, img_transformed)


In [ ]:

# Example 
input_folder = "/content/Grape-harvest-counting/datasets/grapes/test"
output_folder = "/content/Grape-harvest-counting/datasets/grapes_augmented/test"
os.makedirs(output_folder, exist_ok=True)

all_files = os.listdir(input_folder)
img_files = [f for f in all_files if f.endswith(".jpg")]

for img_name in img_files:
    input_image_path = os.path.join(input_folder, img_name)
    output_image_path = os.path.join(output_folder, img_name)
    improve_brightness_saturation(input_image_path, output_image_path)
